In [1]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import splu

In [2]:
mi = 6
nj  = 5

# Initialize matrices with zeros
BI = np.zeros((nj, mi))
AI = np.zeros((mi, nj))
AC = np.zeros((mi, nj))
AD = np.zeros((mi, nj))
BD = np.zeros((nj, mi))

rows = (mi - 2) * (nj - 2)
columns = (mi - 2) * (nj - 2)
A = np.zeros((rows, columns))

# Define other parameters
delta_x = 1.0 / (mi - 1)
delta_y = 1.0 / (nj - 1)
cond_ter = 100.0
temp_ini = 308.0
temp_fin = 298.0
flux_aba = 308.0
flux_arr = 308.0

In [3]:
def ensambla_tdmax(AI, AC, AD, deltax, deltay, cond_ter, temp_ini, temp_fin, jj):
    # Ensure AI, AC, AD are numpy arrays for efficient numerical operations

    # Define boundary conditions
    AI[0, jj] = -temp_ini * cond_ter / (deltax * deltax)
    AD[mi - 1, jj] = -temp_fin * cond_ter / (deltax * deltax)

    # Assemble the tridiagonal matrix and the result vector
    for iin in range(1, mi - 1):
        AI[iin, jj] = -1.0 * cond_ter / (deltax * deltax)
        AC[iin, jj] = 2.0 * cond_ter * (1.0 / (deltax * deltax) + 1.0 / (deltay * deltay))
        AD[iin, jj] = -1.0 * cond_ter / (deltax * deltax)

In [4]:
def ensambla_tdmay(BI, BD, deltax, deltay, cond_ter, flux_aba, flux_arr, ii):
    # Ensure BI and BD are numpy arrays for efficient numerical operations

    # Define boundary conditions
    BI[0, ii] = -flux_aba * cond_ter / (deltay * deltay)
    BD[nj - 1, ii] = -flux_arr * cond_ter / (deltay * deltay)

    # Assemble the tridiagonal matrix and the result vector
    for jjn in range(1, nj - 1):  # Loop from 1 to nj-2
        BI[jjn, ii] = -1.0 * cond_ter / (deltay * deltay)
        BD[jjn, ii] = -1.0 * cond_ter / (deltay * deltay)

In [5]:
def build_global_matrix(A, BI, AI, AC, AD, BD):
    for ii in range(1, mi - 1):

        for jj in range(1, nj - 1):

            if jj >= 2 and jj < nj - 1:
                A[ii - 1, jj + nj - 3] = BI[jj - 1, ii]
            
            if ii >= 2 and ii < mi - 1:
                A[ii - 1, jj - 2] = AI[ii - 1, jj]

            A[ii - 1, jj - 1] = AC[ii, jj]

            if (ii >= 1 and ii < mi - 2):
                A[ii - 1, jj] = AD[ii + 1, jj]
            
            if (jj >= 1 and jj < nj - 2):
                A[ii - 1, jj - nj - 1] = BD[jj + 1, ii]

In [12]:
# Call the function
for jj in range(1, nj - 1):
    ensambla_tdmax(AI, AC, AD, delta_x, delta_y, cond_ter, temp_ini, temp_fin, jj)

for ii in range(1, mi - 1):
    ensambla_tdmay(BI, BD, delta_x, delta_y, cond_ter, flux_aba, flux_arr, ii)

# build_global_matrix(A, BI, AI, AC, AD, BD)
# print(BI)
# print(AI)
# print(AC)
# print(AD)
# print(BD)
# print(A)